# High-Performance Pandas: `eval` and `query`

* While abstractions in PyData stack are efficient and effective for many use cases, they often rely on creation of temporary intermediate objects, which can cause undue overhead in computational time and memory use.
    * To address this, Pandas includes some methods that allow you to directly access C-speed operations w/o costly allocation of intermediate arrays: `eval` and `query`

# A. Motivating `query` and `eval`: Compound Expressions

* `numexpr` evaluates expression in a way that avoids temporary arrays where possible, and thus can be much more efficient than NumPy, especially for long sequences of computations of large arrays.

In [1]:
import numpy as np
rng = np.random.default_rng(42)
x = rng.random(1000000)
y = rng.random(1000000)
%timeit x + y

5.35 ms ± 99.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

207 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit mask = (x > 0.5) & (y < 0.5)

4.08 ms ± 147 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
import numexpr

In [6]:
mask_numexpr = numexpr.evaluate('(x>0.5) & (y<0.5)')
np.all(mask == mask_numexpr)

True

In [7]:
%timeit mask_numexpr

31.7 ns ± 3.39 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


# B. `pandas.eval` for Efficient Operations:

* The `eval` function in Pandas uses string expressions to efficiently compute operations on DataFrame objects.

In [8]:
import pandas as pd
nrows, ncols = 100000, 100
df1, df2, df3, df4 = (pd.DataFrame(rng.random((nrows, ncols))) for i in range(4))

In [9]:
%timeit df1 + df2 + df3 + df4

121 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit pd.eval('df1 + df2 + df3 + df4')

62.5 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
np.allclose(df1 + df2 + df3 + df4, pd.eval('df1 + df2 + df3 + df4'))

True

In [12]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.integers(0, 1000, (100, 3))) for i in range(5))

In [13]:
# Aritmetic Operators

result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

In [14]:
# Comparison Operators

result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

In [15]:
# Bitwise operators

result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [16]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

In [17]:
# Object Attributes and Indices

result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

# C. `DataFrame.eval` for Column-Wise Operations

In [18]:
df = pd.DataFrame(rng.random((1000, 3)), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.850888,0.966709,0.958690
1,0.820126,0.385686,0.061402
2,0.059729,0.831768,0.652259
3,0.244774,0.140322,0.041711
4,0.818205,0.753384,0.578851


In [19]:
df.shape

(1000, 3)

In [20]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [21]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

### C.1. Assignment in DataFrame.eval:

* DataFrame.eval also allows assignment to
any column.

In [22]:
df.head()

,A,B,C
0,0.850888,0.966709,0.958690
1,0.820126,0.385686,0.061402
2,0.059729,0.831768,0.652259
3,0.244774,0.140322,0.041711
4,0.818205,0.753384,0.578851


In [23]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.850888,0.966709,0.958690,1.895916
1,0.820126,0.385686,0.061402,19.638139
2,0.059729,0.831768,0.652259,1.366782
3,0.244774,0.140322,0.041711,9.232370
4,0.818205,0.753384,0.578851,2.715013


In [24]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.850888,0.966709,0.958690,-0.120812
1,0.820126,0.385686,0.061402,7.075399
2,0.059729,0.831768,0.652259,-1.183638
3,0.244774,0.140322,0.041711,2.504142
4,0.818205,0.753384,0.578851,0.111982


### C.2. Local Variables in `DataFrame.eval`

* The `@` character marksa variable name rather than a column name, and lets you efficiently evaluate expressions involving the 2 namespaces: The namespace of the columns, and the namespace of python objects.
* This `@` character is only supported by the `DataFrame.eval` method, not by the `pandas.eval` function, because the `pandas.eval` function only has access to the one (Python) namespace.

In [25]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

# D. The `DataFrame.query` Method:

In [26]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

In [27]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [28]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

# E. Performance: When to Use These Functions:

* When considering whether to use eval and query, there are two considerations: computation
time and memory use. Memory use is the most predictable aspect. 
    * Every compound expression involving NumPy arrays or Pandas Data
Frames will result in implicit creation of temporary arrays
    * The traditional method is faster for smaller arrays

* If the size of the temporary DataFrames is significant compared to your available system
memory (typically several gigabytes), then it’s a good idea to use an eval or
query expression.

In [29]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

# is roughly equivalent to this:

tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

In [30]:
# Check the approximate size of your array in bytes
df.values.nbytes

32000